In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import pandas as pd
import numpy as np

os.getcwd()

'/content'

In [0]:
!apt-get -qq install --no-install-recommends nvidia-375
!apt-get -qq install --no-install-recommends nvidia-opencl-icd-375 nvidia-opencl-dev opencl-headers
!apt-get -qq install --no-install-recommends git cmake build-essential libboost-dev libboost-system-dev libboost-filesystem-dev
!pip3 install -qq lightgbm --install-option=--gpu

Selecting previously unselected package nvidia-384.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../nvidia-384_418.87.00-0ubuntu1_amd64.deb ...
Unpacking nvidia-384 (418.87.00-0ubuntu1) ...
Selecting previously unselected package nvidia-375.
Preparing to unpack .../nvidia-375_384.111-0ubuntu1_amd64.deb ...
Unpacking nvidia-375 (384.111-0ubuntu1) ...
Setting up nvidia-384 (418.87.00-0ubuntu1) ...
Setting up nvidia-375 (384.111-0ubuntu1) ...
Selecting previously unselected package nvidia-headless-no-dkms-430.
(Reading database ... 131189 files and directories currently installed.)
Preparing to unpack .../0-nvidia-headless-no-dkms-430_430.40-0ubuntu0~gpu18.04.1_amd64.deb ...
Unpacking nvidia-headless-no-dkms-430 (430.40-0ubuntu0~gpu18.04.1) ...
Selecting previously unselected package nvidia-headless-430.
Preparing to unpack .../1-nvidia-headless-430_430.40-0ubuntu0~gpu18.04.1_amd64.deb ...
Unpacking nvidia-headless-430 (430.40-0ubuntu0~gpu1

In [0]:
os.chdir('drive/My Drive/bigcontest2019/scripts/model/BJW/')

In [0]:
train = pd.read_csv('Final/train_char_small.csv').reset_index(drop=True)
test1 = test1 = pd.read_csv('Final/train_char_small.csv').reset_index(drop=True)
# test2 = test2 = pd.read_csv('Final/train_char.csv').reset_index(drop=True)
train_label = pd.read_csv('Final/train_label_char.csv').reset_index(drop=True)

In [0]:
# model
from sklearn.model_selection import KFold
import warnings

import os
import gc

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import lightgbm as lgb

In [0]:
FOLDS=3;SEED=42

def lgb_model(train, test, label, folds=FOLDS, seed=SEED):
    
    # skf = StratifiedKFold(n_splits=folds, random_state=seed)
    kf = KFold(n_splits=folds, random_state=seed)
    
    oof_time = np.zeros(len(train))
    oof_spent = np.zeros(len(train))
    pred_time = np.zeros(len(test))
    pred_spent = np.zeros(len(test))
    
    for trn_idx, val_idx in kf.split(train):
        params = {
                'objective':'regression',
                "boosting": "gbdt",
                "num_iterations ":100,
                'num_leaves': 10000,
                'max_depth': 8,
                'learning_rate': 0.1,
                'min_data_in_leaf': 32, 
                'min_child_samples': 30,
                'min_child_weight': 0.5,
                'min_split_gain': 0.005,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.9 ,
                "bagging_freq": 2,
                "bagging_seed": 42,
                "metric": 'rmse',
                "lambda_l1": 0.1,
                "lambda_l2": 0.1,
                "device" : "gpu"
            }
        train_df = lgb.Dataset(train.drop(columns='char_id').loc[trn_idx], label=label.loc[trn_idx, 'amount_spent'])
        valid_df = lgb.Dataset(train.drop(columns='char_id').loc[val_idx], label=label.loc[val_idx, 'amount_spent'])
        
        lgb_model = lgb.train(params, train_df, 5000, valid_sets = [train_df, valid_df], early_stopping_rounds = 100, verbose_eval=5000)
        oof_spent[val_idx] = lgb_model.predict(train.loc[val_idx])/skf.n_splits
        pred_spent += lgb_model.predict(test)/kf.n_splits
      
    # for trn_idx, val_idx in skf.split(train, label['survival_time']):
        
    #     train_df = lgb.Dataset(train.drop(columns='char_id').loc[trn_idx], label=label.loc[trn_idx, 'survival_time'])
    #     valid_df = lgb.Dataset(train.drop(columns='char_id').loc[val_idx], label=label.loc[val_idx, 'survival_time'])
        
    #     lgb_model = lgb.train(params, train_df, 5000, valid_sets = [train_df, valid_df], early_stopping_rounds = 100, verbose_eval=5000)
    #     oof_time[val_idx] = lgb_model.predict(train.loc[val_idx])/skf.n_splits
    #     pred_time += lgb_model.predict(test)/skf.n_splits
    
    return  oof_spent, pred_spent #oof_time,  pred_time,

In [0]:
oof_spent, pred_spent  = lgb_model(train = train,test=test1,label = train_label)

LightGBMError: ignored

In [0]:
train_label.head()

,acc_id,char_id,survival_time,amount_spent
0,2,233598,64,0.00000
1,5,22260,60,0.00000
2,5,175813,60,0.00000
3,5,321231,60,0.00000
4,8,43437,64,0.02031
